In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

21/08/15 18:57:39 WARN Utils: Your hostname, DESKTOP-C98VC8U resolves to a loopback address: 127.0.1.1; using 172.22.196.85 instead (on interface eth0)
21/08/15 18:57:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
21/08/15 18:57:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/08/15 18:57:43 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
customers = spark.read.parquet("./output/customers.parquet")
customers.show(3)

+----------------+---------+--------+--------------------+--------+--------------+
|CustomerIdentity|FirstName|LASTNAME|               phone|postcode|Customer_Title|
+----------------+---------+--------+--------------------+--------+--------------+
|               1|    Terri| Leonard|          3124065463|   65369|          Mrs.|
|               2|  Maurice|    West|001-468-075-5301x308|   27061|           Dr.|
|               3|   Taylor|    Hall|    001-004-938-2651|   88302|           Mr.|
+----------------+---------+--------+--------------------+--------+--------------+
only showing top 3 rows



In [3]:
tickets = spark.read.parquet("./output/tickets.parquet")
tickets.show(3)

+---------+--------+-----------+--------+---------+----------+------------------+------------+----------+
|ticket_id|order_id|customer_id|quantity|Net_sales|event_code|        event_name|event_season|      Date|
+---------+--------+-----------+--------+---------+----------+------------------+------------+----------+
|        1|     182|        160|       1|    58.55|   CHL-ARS|Chelsea vs Arsenal|   2020/2021|2018-05-05|
|        2|     376|         46|       1|    44.25|   CHL-ARS|Chelsea vs Arsenal|   2020/2021|2018-05-05|
|        3|     282|        116|       5|     47.0|   CHL-ARS|Chelsea vs Arsenal|   2020/2021|2018-05-05|
+---------+--------+-----------+--------+---------+----------+------------------+------------+----------+
only showing top 3 rows



In [4]:
#broadcast joining because one table is very small, reduces shuffling
import pyspark.sql.functions as F
join_condition = customers.CustomerIdentity == tickets.customer_id
combined_df =  tickets.join(F.broadcast(customers), join_condition).drop("customer_id")
combined_df.count()

1000

In [5]:
combined_df.printSchema()

root
 |-- ticket_id: integer (nullable = true)
 |-- order_id: integer (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- Net_sales: double (nullable = true)
 |-- event_code: string (nullable = true)
 |-- event_name: string (nullable = true)
 |-- event_season: string (nullable = true)
 |-- Date: date (nullable = true)
 |-- CustomerIdentity: long (nullable = true)
 |-- FirstName: string (nullable = true)
 |-- LASTNAME: string (nullable = true)
 |-- phone: string (nullable = true)
 |-- postcode: string (nullable = true)
 |-- Customer_Title: string (nullable = true)



In [6]:
combined_df.show(1)

+---------+--------+--------+---------+----------+------------------+------------+----------+----------------+---------+--------+--------------------+--------+--------------+
|ticket_id|order_id|quantity|Net_sales|event_code|        event_name|event_season|      Date|CustomerIdentity|FirstName|LASTNAME|               phone|postcode|Customer_Title|
+---------+--------+--------+---------+----------+------------------+------------+----------+----------------+---------+--------+--------------------+--------+--------------+
|        1|     182|       1|    58.55|   CHL-ARS|Chelsea vs Arsenal|   2020/2021|2018-05-05|             160|     Tony|   Baird|001-097-596-0629x...|   94414|           Ms.|
+---------+--------+--------+---------+----------+------------------+------------+----------+----------------+---------+--------+--------------------+--------+--------------+
only showing top 1 row



In [7]:
combined_df.select("Date").where(F.col("Date").isNull()).count()

0

In [8]:
import pyspark.sql.functions as F
# 3. For each Customer, a list of Events
Edf = combined_df.groupBy("CustomerIdentity").agg(F.collect_list(F.col("event_code")).alias("Events_list"))
Edf.show(5, truncate=False)

+----------------+------------------------------------+
|CustomerIdentity|Events_list                         |
+----------------+------------------------------------+
|26              |[BTH-SRC, BTH-SRC, CHL-ARS, BTH-SRC]|
|29              |[F1-SILV, CHL-ARS, F1-SILV]         |
|65              |[F1-SILV, BTH-SRC, CHL-ARS]         |
|191             |[CHL-ARS, F1-SILV, CHL-ARS, CHL-ARS]|
|19              |[CHL-ARS, F1-SILV, CHL-ARS]         |
+----------------+------------------------------------+
only showing top 5 rows



In [9]:
# combined_df.groupBy("CustomerIdentity").agg(F.count("event_code").alias("Totals")).sort("Totals", ascending=True).show()
Tots = combined_df.groupBy("CustomerIdentity").agg(F.count("event_name").alias("Totals"))
Tots.show(2)

+----------------+------+
|CustomerIdentity|Totals|
+----------------+------+
|              26|     4|
|              29|     3|
+----------------+------+
only showing top 2 rows



In [26]:
MultiEvent = Tots.withColumn("MultiEvent", F.col("Totals") > 1)
MultiEvent.show()

TypeError: 'Column' object is not callable

In [11]:
def multiEvent(x):
    
   if x > 1:

        return   "True"
   else:
        return " "
    

In [13]:
from pyspark.sql.functions import udf
from pyspark.sql.types import *
multiEvent_udf = udf(multiEvent, StringType())

In [14]:
#List of **all** Customers with an additional column called "MultiEvent", set to `True` for those Customers with more than 1 Event
Tots.withColumn("MultiEvent", multiEvent_udf(F.col("Totals"))).show(5)

+----------------+------+----------+
|CustomerIdentity|Totals|MultiEvent|
+----------------+------+----------+
|              26|     4|      True|
|              29|     3|      True|
|              65|     3|      True|
|             191|     4|      True|
|              54|     8|      True|
|              19|     3|      True|
|             167|     3|      True|
|             112|     5|      True|
|             113|     3|      True|
|             155|     4|      True|
|             198|     5|      True|
|              22|     3|      True|
|             196|     6|      True|
|             130|     3|      True|
|              77|     9|      True|
|               7|     4|      True|
|              34|     8|      True|
|             184|     3|      True|
|             188|     6|      True|
|             126|     1|          |
|              50|     6|      True|
|              94|     3|      True|
|             149|     1|          |
|             190|     4|      True|
|

In [15]:
from pyspark.sql import Window

WindowSpec = Window.partitionBy("CustomerIdentity")

df22 = combined_df.withColumn("EventsList", F.collect_list(F.col("event_code")).over(WindowSpec)).select("CustomerIdentity", "EventsList")
df22.show(5, truncate=False)



+----------------+------------------------------------+
|CustomerIdentity|EventsList                          |
+----------------+------------------------------------+
|26              |[BTH-SRC, BTH-SRC, BTH-SRC, CHL-ARS]|
|26              |[BTH-SRC, BTH-SRC, BTH-SRC, CHL-ARS]|
|26              |[BTH-SRC, BTH-SRC, BTH-SRC, CHL-ARS]|
|26              |[BTH-SRC, BTH-SRC, BTH-SRC, CHL-ARS]|
|29              |[F1-SILV, F1-SILV, CHL-ARS]         |
+----------------+------------------------------------+
only showing top 5 rows



In [16]:
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol="event_code", outputCol="IndexEvents")
indexed = indexer.fit(combined_df).transform(combined_df)
indexed.show(10)

+---------+--------+--------+---------+----------+------------------+------------+----------+----------------+---------+--------+--------------------+--------+--------------+-----------+
|ticket_id|order_id|quantity|Net_sales|event_code|        event_name|event_season|      Date|CustomerIdentity|FirstName|LASTNAME|               phone|postcode|Customer_Title|IndexEvents|
+---------+--------+--------+---------+----------+------------------+------------+----------+----------------+---------+--------+--------------------+--------+--------------+-----------+
|        1|     182|       1|    58.55|   CHL-ARS|Chelsea vs Arsenal|   2020/2021|2018-05-05|             160|     Tony|   Baird|001-097-596-0629x...|   94414|           Ms.|        0.0|
|        2|     376|       1|    44.25|   CHL-ARS|Chelsea vs Arsenal|   2020/2021|2018-05-05|              46|   Justin| Higgins|          1889149702|   67579|          Mrs.|        0.0|
|        3|     282|       5|     47.0|   CHL-ARS|Chelsea vs Arse

In [17]:
combined_df.groupBy("Customer_Title").agg(F.sum("quantity").alias("Totals")).sort("Totals", ascending=False).show()

+--------------+------+
|Customer_Title|Totals|
+--------------+------+
|           Dr.|  1023|
|           Mr.|   996|
|          Mrs.|   691|
|          Miss|   131|
|           Ms.|   128|
+--------------+------+



In [18]:
#List of **all** Customers with an additional column called "MultiEvent", set to `True` for those Customers with more than 1 Event
combined_df.groupBy("CustomerIdentity").agg(F.count("event_code").alias("Totals")).sort("Totals", ascending=True).show()

+----------------+------+
|CustomerIdentity|Totals|
+----------------+------+
|              86|     1|
|             175|     1|
|              28|     1|
|             121|     1|
|             126|     1|
|             149|     1|
|              53|     1|
|              61|     2|
|              32|     2|
|              18|     2|
|             145|     2|
|             156|     2|
|             154|     2|
|              13|     2|
|               1|     2|
|             123|     2|
|              71|     2|
|             144|     2|
|             171|     2|
|              66|     2|
+----------------+------+
only showing top 20 rows



In [19]:
# For each Customer, a list of Events
# combined_df.groupBy("CustomerIdentity").agg("event_name").select("CustomerIdentity", "event_name").show()

In [20]:
combined_df.select("CustomerIdentity").distinct().count()

198

In [21]:
combined_df.selectExpr("`Net_sales` > 2").show(3)

+---------------+
|(Net_sales > 2)|
+---------------+
|           true|
|           true|
|           true|
+---------------+
only showing top 3 rows



In [22]:
import pyspark.sql.functions as F 
event_df = indexed.groupby("CustomerIdentity").agg(F.collect_list("IndexEvents").alias("List_of_events"))
event_df.show(3, truncate=False)

+----------------+--------------------+
|CustomerIdentity|List_of_events      |
+----------------+--------------------+
|26              |[1.0, 1.0, 0.0, 1.0]|
|29              |[2.0, 0.0, 2.0]     |
|65              |[2.0, 1.0, 0.0]     |
+----------------+--------------------+
only showing top 3 rows

